In [ ]:
#Example
windowsize =3
listA = [3,2,5,9,4,6,3,8,7,9]
listB=[max(listA[i:i+ windowsize]) for i in range(0,len(listA), windowsize)]
print (listB)


In [ ]:
#Example
max1 = [max(listA[i:i+ windowsize]) for i in range(0, len(listA), windowsize)]
print(max1)

In [ ]:
#use this
for i in range(0, len(listA),  windowsize):
    max_frame1 = list(listA[i:i+ windowsize])
    print('Jumping window', max_frame1)
    max_frame2 = max(max_frame1)
    print('max_frame', max_frame2)

In [ ]:
#GoP based Token bucket

listA = [3,2,5,9,4,6,3,8,7,9]
k=0
for i in range(0, len(listA), windowsize):
    GoP_pattern = list(listA[i:i+windowsize])
    print('GoP based pattern', GoP_pattern)
    
    GoP_mean = sum(GoP_pattern)/3
    print('mean', GoP_mean)


In [1]:
#import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder


from scipy.stats.distributions import gamma
from scipy.stats.distributions import poisson
from scipy.stats.distributions import expon

import statistics
from scipy.stats import norm
from scipy import stats
import scipy.stats

import matplotlib.pyplot as plt



In [2]:
#import data
data = pd.read_csv("../GoP based TOKEN BUCKET algorithm/H264-data/nbc_news_G16B7_QP28.csv",infer_datetime_format=True,
                  parse_dates=['arrival_time'], index_col='arrival_time', 
                   names=['decoded_frame','arrival_time', 'video_frame', 
                                         'frame_size', 'x', 'y', 'z'])

In [3]:
data.head()

,decoded_frame,video_frame,frame_size,x,y,z
arrival_time,,,,,,
0.00000,0,I,8752,40.420,45.968,47.559
266.66667,8,P,1096,40.185,46.050,47.023
33.33333,1,B,128,40.107,45.591,45.966
66.66667,2,B,128,40.102,46.047,46.251
100.00000,3,B,128,40.131,45.922,46.705


In [4]:
#drop decoded frame column
data.drop(('decoded_frame'), axis=1, inplace=True)

data.drop(('x'), axis=1, inplace=True)
data.drop(('y'), axis=1, inplace=True)
data.drop(('z'), axis=1, inplace=True)

In [5]:
# Label encoding of species column numerically
le = LabelEncoder()
le.fit(data['video_frame'])
print(list(le.classes_))
data['video_frame'] = le.transform(data['video_frame'])

['B', 'I', 'P']


In [6]:
data.head()

,video_frame,frame_size
arrival_time,,
0.00000,1,8752
266.66667,2,1096
33.33333,0,128
66.66667,0,128
100.00000,0,128


In [7]:
data.frame_size.dropna(0,inplace= True)

In [8]:
data.reset_index(inplace=True)

In [9]:
data.head()

,arrival_time,video_frame,frame_size
0,0.00000,1,8752
1,266.66667,2,1096
2,33.33333,0,128
3,66.66667,0,128
4,100.00000,0,128


In [10]:
#Sort the arrival time

data.sort_values('arrival_time', inplace=True)
data.reset_index(drop=True, inplace=True)

In [11]:
data.head()

,arrival_time,video_frame,frame_size
0,0.00000,1,8752
1,33.33333,0,128
2,66.66667,0,128
3,100.00000,0,128
4,133.33333,0,128


In [12]:
#Check missing values

data.isnull().sum()

arrival_time    0
video_frame     0
frame_size      0
dtype: int64

In [13]:
#Define the parameters

peak_frame = max(data.arrival_time)
print ('peak_frame', peak_frame)

mean_val = np.mean(data.frame_size)
var_val = np.var(data.frame_size)
print('mean_val', mean_val)
print('var_val', var_val)

alpha = mean_val**2 / var_val
beta = var_val / mean_val
print('alpha', alpha)
print('beta', beta)

peak_frame 1650666.667
mean_val 15080.51436764201
var_val 467122350.54288137
alpha 0.48685727267888457
beta 30975.2266504369


In [14]:
mean_frame = np.mean(data.frame_size)
mean_frame = np.ceil(mean_frame)
print('mean_frame', mean_frame)

num_of_frames = data.shape[0]
total = np.sum(data.frame_size)
print(num_of_frames)
print(total)

mean_frame 15081.0
49521
746802152


In [15]:
len(data.frame_size)

49521

In [16]:
data.shape[0]

49521

In [17]:
data.head(20)

,arrival_time,video_frame,frame_size
0,0.00000,1,8752
1,33.33333,0,128
2,66.66667,0,128
3,100.00000,0,128
4,133.33333,0,128
5,166.66667,0,128
6,200.00000,0,128
7,233.33333,0,128
8,266.66667,2,1096
9,300.00000,0,312


***window size = 5***

In [18]:
#Final Jumping window algorithm

windowSize = 5
addition =0
max_frame= 0
k= 0

for i in range(0, len(data['frame_size']),  windowSize):
    byte_sent = 0
    deleted_bytes = 0
    bucket_size = 0
    marked_bytes = 0
    #j = 0
    #l = 0
    
    #Define GoP pattern
    if(k < len(data['frame_size']) and k+16 < len(data['frame_size'])):
        GoP_pattern = data['frame_size'][k:k+16]
        k += 16
        GoP_mean = sum(GoP_pattern)/16
    else:
        GoP_mean = sum(data['frame_size'][0:16])
    
    #print(GoP_pattern)
    #print(GoP_mean)
    
    max_bucket_size = windowSize * GoP_mean
    
    for j in range(i, i+windowSize):
        
        bucket_size += mean_frame
        
        if bucket_size > max_bucket_size:
            bucket_size = max_bucket_size
        
        
        if data.loc[i, 'frame_size'] <= bucket_size:            
            byte_sent += data.loc[i, 'frame_size']
            bucket_size = (bucket_size - data.loc[i, 'frame_size'])
            
        else:
            byte_sent += bucket_size
            marked_bytes += (data.loc[i, 'frame_size'] - bucket_size)
        
            
    addition += abs(marked_bytes)        
            
percent = addition * 100 / total
print(percent)

24.8830804119054


***window size = 10***

In [19]:
#Final Jumping window algorithm

windowSize = 10
addition =0
max_frame= 0
k= 0

for i in range(0, len(data['frame_size']),  windowSize):
    byte_sent = 0
    deleted_bytes = 0
    bucket_size = 0
    marked_bytes = 0
    #j = 0
    #l = 0
    
    #Define GoP pattern
    if(k < len(data['frame_size']) and k+16 < len(data['frame_size'])):
        GoP_pattern = data['frame_size'][k:k+16]
        k += 16
        GoP_mean = sum(GoP_pattern)/16
    else:
        GoP_mean = sum(data['frame_size'][0:16])
    
    #print(GoP_pattern)
    #print(GoP_mean)
    
    max_bucket_size = windowSize * GoP_mean
    
    for j in range(i, i+windowSize):
        
        bucket_size += mean_frame
        
        if bucket_size > max_bucket_size:
            bucket_size = max_bucket_size
        
        
        if data.loc[i, 'frame_size'] <= bucket_size:            
            byte_sent += data.loc[i, 'frame_size']
            bucket_size = (bucket_size - data.loc[i, 'frame_size'])
            
        else:
            byte_sent += bucket_size
            marked_bytes += (data.loc[i, 'frame_size'] - bucket_size)
        
            
    addition += abs(marked_bytes)        
            
percent = addition * 100 / total
print(percent)

36.64237311946043


***window size = 15***

In [20]:
#Final Jumping window algorithm

windowSize = 15
addition =0
max_frame= 0
k= 0

for i in range(0, len(data['frame_size']),  windowSize):
    byte_sent = 0
    deleted_bytes = 0
    bucket_size = 0
    marked_bytes = 0
    #j = 0
    #l = 0
    
    #Define GoP pattern
    if(k < len(data['frame_size']) and k+16 < len(data['frame_size'])):
        GoP_pattern = data['frame_size'][k:k+16]
        k += 16
        GoP_mean = sum(GoP_pattern)/16
    else:
        GoP_mean = sum(data['frame_size'][0:16])
    
    #print(GoP_pattern)
    #print(GoP_mean)
    
    max_bucket_size = windowSize * GoP_mean
    
    for j in range(i, i+windowSize):
        
        bucket_size += mean_frame
        
        if bucket_size > max_bucket_size:
            bucket_size = max_bucket_size
        
        
        if data.loc[i, 'frame_size'] <= bucket_size:            
            byte_sent += data.loc[i, 'frame_size']
            bucket_size = (bucket_size - data.loc[i, 'frame_size'])
            
        else:
            byte_sent += bucket_size
            marked_bytes += (data.loc[i, 'frame_size'] - bucket_size)
        
            
    addition += abs(marked_bytes)        
            
percent = addition * 100 / total
print(percent)

18.93376888662206


***window size = 20***

In [21]:
#Final Jumping window algorithm

windowSize = 20
addition =0
max_frame= 0
k= 0

for i in range(0, len(data['frame_size']),  windowSize):
    byte_sent = 0
    deleted_bytes = 0
    bucket_size = 0
    marked_bytes = 0
    #j = 0
    #l = 0
    
    #Define GoP pattern
    if(k < len(data['frame_size']) and k+16 < len(data['frame_size'])):
        GoP_pattern = data['frame_size'][k:k+16]
        k += 16
        GoP_mean = sum(GoP_pattern)/16
    else:
        GoP_mean = sum(data['frame_size'][0:16])
    
    #print(GoP_pattern)
    #print(GoP_mean)
    
    max_bucket_size = windowSize * GoP_mean
    
    for j in range(i, i+windowSize):
        
        bucket_size += mean_frame
        
        if bucket_size > max_bucket_size:
            bucket_size = max_bucket_size
        
        
        if data.loc[i, 'frame_size'] <= bucket_size:            
            byte_sent += data.loc[i, 'frame_size']
            bucket_size = (bucket_size - data.loc[i, 'frame_size'])
            
        else:
            byte_sent += bucket_size
            marked_bytes += (data.loc[i, 'frame_size'] - bucket_size)
        
            
    addition += abs(marked_bytes)        
            
percent = addition * 100 / total
print(percent)

60.56520000494053


***window size = 25***

In [22]:
#Final Jumping window algorithm

windowSize = 25
addition =0
max_frame= 0
k= 0

for i in range(0, len(data['frame_size']),  windowSize):
    byte_sent = 0
    deleted_bytes = 0
    bucket_size = 0
    marked_bytes = 0
    #j = 0
    #l = 0
    
    #Define GoP pattern
    if(k < len(data['frame_size']) and k+16 < len(data['frame_size'])):
        GoP_pattern = data['frame_size'][k:k+16]
        k += 16
        GoP_mean = sum(GoP_pattern)/16
    else:
        GoP_mean = sum(data['frame_size'][0:16])
    
    #print(GoP_pattern)
    #print(GoP_mean)
    
    max_bucket_size = windowSize * GoP_mean
    
    for j in range(i, i+windowSize):
        
        bucket_size += mean_frame
        
        if bucket_size > max_bucket_size:
            bucket_size = max_bucket_size
        
        
        if data.loc[i, 'frame_size'] <= bucket_size:            
            byte_sent += data.loc[i, 'frame_size']
            bucket_size = (bucket_size - data.loc[i, 'frame_size'])
            
        else:
            byte_sent += bucket_size
            marked_bytes += (data.loc[i, 'frame_size'] - bucket_size)
        
            
    addition += abs(marked_bytes)        
            
percent = addition * 100 / total
print(percent)

17.508846975041926


***window size = 30***

In [23]:
#Final Jumping window algorithm

windowSize = 30
addition =0
max_frame= 0
k= 0

for i in range(0, len(data['frame_size']),  windowSize):
    byte_sent = 0
    deleted_bytes = 0
    bucket_size = 0
    marked_bytes = 0
    #j = 0
    #l = 0
    
    #Define GoP pattern
    if(k < len(data['frame_size']) and k+16 < len(data['frame_size'])):
        GoP_pattern = data['frame_size'][k:k+16]
        k += 16
        GoP_mean = sum(GoP_pattern)/16
    else:
        GoP_mean = sum(data['frame_size'][0:16])
    
    #print(GoP_pattern)
    #print(GoP_mean)
    
    max_bucket_size = windowSize * GoP_mean
    
    for j in range(i, i+windowSize):
        
        bucket_size += mean_frame
        
        if bucket_size > max_bucket_size:
            bucket_size = max_bucket_size
        
        
        if data.loc[i, 'frame_size'] <= bucket_size:            
            byte_sent += data.loc[i, 'frame_size']
            bucket_size = (bucket_size - data.loc[i, 'frame_size'])
            
        else:
            byte_sent += bucket_size
            marked_bytes += (data.loc[i, 'frame_size'] - bucket_size)
        
            
    addition += abs(marked_bytes)        
            
percent = addition * 100 / total
print(percent)

31.710797078688653


***window size = 35***

In [24]:
#Final Jumping window algorithm

windowSize = 35
addition =0
max_frame= 0
k= 0

for i in range(0, len(data['frame_size']),  windowSize):
    byte_sent = 0
    deleted_bytes = 0
    bucket_size = 0
    marked_bytes = 0
    #j = 0
    #l = 0
    
    #Define GoP pattern
    if(k < len(data['frame_size']) and k+16 < len(data['frame_size'])):
        GoP_pattern = data['frame_size'][k:k+16]
        k += 16
        GoP_mean = sum(GoP_pattern)/16
    else:
        GoP_mean = sum(data['frame_size'][0:16])
    
    #print(GoP_pattern)
    #print(GoP_mean)
    
    max_bucket_size = windowSize * GoP_mean
    
    for j in range(i, i+windowSize):
        
        bucket_size += mean_frame
        
        if bucket_size > max_bucket_size:
            bucket_size = max_bucket_size
        
        
        if data.loc[i, 'frame_size'] <= bucket_size:            
            byte_sent += data.loc[i, 'frame_size']
            bucket_size = (bucket_size - data.loc[i, 'frame_size'])
            
        else:
            byte_sent += bucket_size
            marked_bytes += (data.loc[i, 'frame_size'] - bucket_size)
        
            
    addition += abs(marked_bytes)        
            
percent = addition * 100 / total
print(percent)

17.34270859465868


***window size = 40***

In [25]:
#Final Jumping window algorithm

windowSize = 40
addition =0
max_frame= 0
k= 0

for i in range(0, len(data['frame_size']),  windowSize):
    byte_sent = 0
    deleted_bytes = 0
    bucket_size = 0
    marked_bytes = 0
    #j = 0
    #l = 0
    
    #Define GoP pattern
    if(k < len(data['frame_size']) and k+16 < len(data['frame_size'])):
        GoP_pattern = data['frame_size'][k:k+16]
        k += 16
        GoP_mean = sum(GoP_pattern)/16
    else:
        GoP_mean = sum(data['frame_size'][0:16])
    
    #print(GoP_pattern)
    #print(GoP_mean)
    
    max_bucket_size = windowSize * GoP_mean
    
    for j in range(i, i+windowSize):
        
        bucket_size += mean_frame
        
        if bucket_size > max_bucket_size:
            bucket_size = max_bucket_size
        
        
        if data.loc[i, 'frame_size'] <= bucket_size:            
            byte_sent += data.loc[i, 'frame_size']
            bucket_size = (bucket_size - data.loc[i, 'frame_size'])
            
        else:
            byte_sent += bucket_size
            marked_bytes += (data.loc[i, 'frame_size'] - bucket_size)
        
            
    addition += abs(marked_bytes)        
            
percent = addition * 100 / total
print(percent)

111.36288785091772


***window size = 45***

In [26]:
#Final Jumping window algorithm

windowSize = 45
addition =0
max_frame= 0
k= 0

for i in range(0, len(data['frame_size']),  windowSize):
    byte_sent = 0
    deleted_bytes = 0
    bucket_size = 0
    marked_bytes = 0
    #j = 0
    #l = 0
    
    #Define GoP pattern
    if(k < len(data['frame_size']) and k+16 < len(data['frame_size'])):
        GoP_pattern = data['frame_size'][k:k+16]
        k += 16
        GoP_mean = sum(GoP_pattern)/16
    else:
        GoP_mean = sum(data['frame_size'][0:16])
    
    #print(GoP_pattern)
    #print(GoP_mean)
    
    max_bucket_size = windowSize * GoP_mean
    
    for j in range(i, i+windowSize):
        
        bucket_size += mean_frame
        
        if bucket_size > max_bucket_size:
            bucket_size = max_bucket_size
        
        
        if data.loc[i, 'frame_size'] <= bucket_size:            
            byte_sent += data.loc[i, 'frame_size']
            bucket_size = (bucket_size - data.loc[i, 'frame_size'])
            
        else:
            byte_sent += bucket_size
            marked_bytes += (data.loc[i, 'frame_size'] - bucket_size)
        
            
    addition += abs(marked_bytes)        
            
percent = addition * 100 / total
print(percent)

17.100252276723488


***window size = 50***

In [27]:
#Final Jumping window algorithm

windowSize = 50
addition =0
max_frame= 0
k= 0

for i in range(0, len(data['frame_size']),  windowSize):
    byte_sent = 0
    deleted_bytes = 0
    bucket_size = 0
    marked_bytes = 0
    #j = 0
    #l = 0
    
    #Define GoP pattern
    if(k < len(data['frame_size']) and k+16 < len(data['frame_size'])):
        GoP_pattern = data['frame_size'][k:k+16]
        k += 16
        GoP_mean = sum(GoP_pattern)/16
    else:
        GoP_mean = sum(data['frame_size'][0:16])
    
    #print(GoP_pattern)
    #print(GoP_mean)
    
    max_bucket_size = windowSize * GoP_mean
    
    for j in range(i, i+windowSize):
        
        bucket_size += mean_frame
        
        if bucket_size > max_bucket_size:
            bucket_size = max_bucket_size
        
        
        if data.loc[i, 'frame_size'] <= bucket_size:            
            byte_sent += data.loc[i, 'frame_size']
            bucket_size = (bucket_size - data.loc[i, 'frame_size'])
            
        else:
            byte_sent += bucket_size
            marked_bytes += (data.loc[i, 'frame_size'] - bucket_size)
        
            
    addition += abs(marked_bytes)        
            
percent = addition * 100 / total
print(percent)

30.122195737861237
